https://ai.google.dev/gemma/docs/core/huggingface_text_full_finetune

https://huggingface.co/collections/google/gemma-3-release

https://huggingface.co/datasets/bebechien/MobileGameNPC

In [1]:
import json
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from random import randint
import re

from trl import SFTConfig, SFTTrainer

import matplotlib.pyplot as plt

/home/florin/git/FlorinAndrei/gemma3-full-finetune-social-media/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


In [ ]:
def create_conversation(sample):
    return {
        "messages": [
            {"role": "user", "content": sample["player"]},
            {"role": "assistant", "content": sample["alien"]},
        ]
    }


npc_type = "martian"

# Load dataset from the Hub
dataset = load_dataset("bebechien/MobileGameNPC", npc_type, split="train")

In [4]:
print(dataset.column_names)
print(json.dumps(dataset[4], indent=2))

['id', 'category', 'player', 'alien']
{
  "id": 5,
  "category": "Greetings & Basic Questions",
  "player": "Goodbye.",
  "alien": "Vorp-vorp, Earth-kin. May your zky stay blue."
}


In [ ]:
# Convert dataset to conversational format
dataset = dataset.map(create_conversation, remove_columns=dataset.features, batched=False)

# Split dataset into 80% training samples and 20% test samples
dataset = dataset.train_test_split(test_size=0.2, shuffle=False)

# Print formatted user prompt
print(json.dumps(dataset["train"][0]["messages"], indent=2))

In [4]:
n_example = 4

dataset2 = load_dataset(
    "csv", name="social-media-conversations", split="train", data_files="conversations.csv"
)
dataset2 = dataset2.shuffle(seed=42)
print(dataset2.column_names)
print(json.dumps(dataset2[n_example], indent=2))

Generating train split: 0 examples [00:00, ? examples/s]

['timestamp', 'comment_id', 'comment_body', 'parent_text']
{
  "timestamp": "2021-08-25 16:32:18 UTC",
  "comment_id": "hab6nka",
  "comment_body": "Pfft, just turn the volume way up, it'll blow dry it.",
  "parent_text": "Can confirm, shoulder length hair is not conducive to wearing headphones for the 4 hours after I shower."
}


In [3]:
base_model = "google/gemma-3-4b-it"
checkpoint_dir = "checkpoints"
learning_rate = 5e-5

In [ ]:
torch_dtype = torch.bfloat16

# Define model init arguments
model_kwargs = dict(
    attn_implementation="eager",  # Use "flash_attention_2" when running on Ampere or newer GPU
    torch_dtype=torch_dtype,  # What torch dtype to use, defaults to auto
    device_map="auto",  # Let torch decide how to load the model
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(base_model, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(base_model)

print(f"Device: {model.device}")
print(f"DType: {model.dtype}")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device: cuda:0
DType: torch.bfloat16


In [5]:
print(model)

Gemma3ForConditionalGeneration(
  (model): Gemma3Model(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(4096, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-26): 27 x SiglipEncoderLayer(
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (self_attn): SiglipAttention(
                (k_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
                (out_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              )
              (layer_norm2): LayerNorm((1152,), eps=

In [ ]:
# Load the model and tokenizer into the pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load a random sample from the test dataset
rand_idx = randint(0, len(dataset["test"]) - 1)
test_sample = dataset["test"][rand_idx]

# Convert as test example into a prompt with the Gemma template
prompt = pipe.tokenizer.apply_chat_template(
    test_sample["messages"][:1], tokenize=False, add_generation_prompt=True
)
outputs = pipe(prompt, max_new_tokens=256, disable_compile=True)

# Extract the user query and original answer
print(f"Question:\n{test_sample['messages'][0]['content']}\n")
print(f"Original Answer:\n{test_sample['messages'][1]['content']}\n")
print(f"Generated Answer (base model):\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Device set to use cuda:0


Question:
(Stands idle for too long)

Original Answer:
You'z broken, Terran? Or iz diz... 'meditation'? You look like you're trying to lay an egg.

Generated Answer (base model):
Okay, you've been standing idle for a while! That's perfectly alright. It happens to the best of us. 

It's a good opportunity to just *be* for a moment. 

Is there anything you'd like to do? Here are some ideas, or we can just let the silence hang for a bit:

*   **Talk about something:** Do you want to chat about anything at all?  A hobby, a recent event, a question you have, anything?
*   **Play a game:** I can play a simple game with you like 20 questions, a number guessing game, or even just tell you a short story.
*   **Ask me a question:**  Do you have any questions for me about anything?
*   **Just be quiet:** Sometimes, silence is nice too.  If you just want to be present, that's perfectly fine.

Just let me know what you'd like to do, or if you'd just like me to wait a little while longer.


In [7]:
torch_dtype = model.dtype

args = SFTConfig(
    output_dir=checkpoint_dir,  # directory to save and repository id
    max_length=512,  # max sequence length for model and packing of the dataset
    packing=False,  # Groups multiple samples in the dataset into a single sequence
    num_train_epochs=5,  # number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    gradient_checkpointing=False,  # Caching is incompatible with gradient checkpointing
    optim="adamw_torch_fused",  # use fused adamw optimizer
    logging_steps=1,  # log every step
    save_strategy="no",  # save checkpoint never
    eval_strategy="epoch",  # evaluate checkpoint every epoch
    learning_rate=learning_rate,  # learning rate
    fp16=True if torch_dtype == torch.float16 else False,  # use float16 precision
    bf16=True if torch_dtype == torch.bfloat16 else False,  # use bfloat16 precision
    lr_scheduler_type="constant",  # use constant learning rate scheduler
    push_to_hub=False,  # do not push model to hub
    report_to="tensorboard",  # report metrics to tensorboard
    dataset_kwargs={
        "add_special_tokens": False,  # Template with special tokens
        "append_concat_token": True,  # Add EOS token as separator token between examples
    },
)

In [8]:
# Create Trainer object
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    processing_class=tokenizer,
)

Tokenizing train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [ ]:
# Start training, the model will be automatically saved to the Hub and the output directory
trainer.train()

In [ ]:
# Save the final model again to the Hugging Face Hub
trainer.save_model()

In [ ]:
# Access the log history
log_history = trainer.state.log_history

# Extract training / validation loss
train_losses = [log["loss"] for log in log_history if "loss" in log]
epoch_train = [log["epoch"] for log in log_history if "loss" in log]
eval_losses = [log["eval_loss"] for log in log_history if "eval_loss" in log]
epoch_eval = [log["epoch"] for log in log_history if "eval_loss" in log]

# Plot the training loss
plt.plot(epoch_train, train_losses, label="Training Loss")
plt.plot(epoch_eval, eval_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss per Epoch")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
del model, tokenizer, pipe, trainer
torch.cuda.empty_cache()

In [ ]:
model_id = checkpoint_dir

# Load Model
model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype="auto", device_map="auto", attn_implementation="eager"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# Load the model and tokenizer into the pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


def test(test_sample):
    # Convert as test example into a prompt with the Gemma template
    prompt = pipe.tokenizer.apply_chat_template(
        test_sample["messages"][:1], tokenize=False, add_generation_prompt=True
    )
    outputs = pipe(prompt, max_new_tokens=256, disable_compile=True)

    # Extract the user query and original answer
    print(f"Question:\n{test_sample['messages'][0]['content']}")
    print(f"Original Answer:\n{test_sample['messages'][1]['content']}")
    print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
    print("-" * 80)


# Test with an unseen dataset
for item in dataset['test']:
    test(item)